In [1]:
!pip install -U -q transformers==4.39.3
!pip install -U -q accelerate==0.28.0
!pip install -U -q datasets==2.18.0
# !pip install -U -q peft==0.10.0
!pip install -U -q bitsandbytes==0.43.1
!pip install -U -q trl==0.8.6

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.6.1 requires cubinlinker, which is not installed.
cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.6.1 requires ptxcompiler, which is not installed.
cuml 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.6.1 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.5.0 which is incompatible.
distributed 2024.5.1 requires dask==2024.5.1, but you have dask 2024.7.0 which is incompatible.
gcsfs 2024.5.0 requires fsspec==2024.5.0, but you have fsspec 2024.2.0 which is incompatible.
rapids-dask-dependency 24.6.0a0 requires dask==2024.5.1, but you have dask 2024.7.0 which is incompatible.
s3fs 2024.5.0 requires fsspec==2024.5.0.*, but you have fsspec 2024.2.0 which is incompatible.
ERROR:

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_W"))

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "Fine-tuning HuggingFace SmolLM-135M-Instruct with simple calculation"
os.environ["WANDB_NAME"] = "ft-smollm-135M-instruct-with-simple-cal"
os.environ["MODEL_NAME"] = "HuggingFaceTB/SmolLM-135M-Instruct"
os.environ["TOKENIZER_NAME"] = "HuggingFaceTB/SmolLM-135M-Instruct"
os.environ["DATASET"] = "micost/simple_calculation_orpo"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from datasets import load_dataset

ds=load_dataset(os.getenv("DATASET"), split=["train"])
ds

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

[Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 50000
 })]

In [5]:
train_ds=ds[0].shuffle(seed=42).select(range(50000))

print(train_ds)

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 50000
})


In [6]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(os.getenv("TOKENIZER_NAME"))

# decoder-only architecture, no need eos token
tokenizer.pad_token=tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/3.59k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

In [7]:
import torch, multiprocessing

def preprocess(x):
    x["chosen"]=tokenizer.apply_chat_template(x["chosen"], tokenize=False)
    x["rejected"]=tokenizer.apply_chat_template(x["rejected"], tokenize=False)
    return x

train_ds=train_ds.map(preprocess, num_proc=multiprocessing.cpu_count(), load_from_cache_file=False)

Map (num_proc=4):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoModelForCausalLM
model=AutoModelForCausalLM.from_pretrained(
    os.getenv("MODEL_NAME"),
    torch_dtype=torch.float16,
#     device_map={"": 0},
    device_map="cuda",
    trust_remote_code=True
)

model.gradient_checkpointing_enable()
model.device

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

device(type='cuda', index=0)

In [9]:
from trl import ORPOTrainer, ORPOConfig

orpo_config=ORPOConfig(
    output_dir=os.getenv("WANDB_NAME"),
    #evaluation_strategy="steps",
    #do_eval=True,
    optim="adamw_8bit",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    #per_device_eval_batch_size=4,
    log_level="debug",
    logging_steps=1000,
    learning_rate=3e-4,
    #eval_steps=100,
    save_steps=1000,
    save_strategy="epoch",
    num_train_epochs=1,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    beta=0.1, # beta is ORPO's lambda
    max_length=1024,
    report_to="tensorboard",
    run_name=os.getenv('WANDB_NAME')
)

trainer = ORPOTrainer(
        model=model,
        train_dataset=train_ds,
        #eval_dataset=eval_ds,
        args=orpo_config,
        tokenizer=tokenizer,
)

trainer.train()

2024-10-29 02:25:42.074781: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 02:25:42.074893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 02:25:42.211842: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
Currently training with a batch size of: 4
***** Running training *****
  Num examples = 50,000
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 6,250
  Number of trainable parameters = 134,515,008
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1000,0.526900
2000,0.286100
3000,0.248600
4000,0.234000
5000,0.220600
6000,0.214200


Saving model checkpoint to ft-smollm-135M-instruct-with-simple-cal/checkpoint-6250
Configuration saved in ft-smollm-135M-instruct-with-simple-cal/checkpoint-6250/config.json
Configuration saved in ft-smollm-135M-instruct-with-simple-cal/checkpoint-6250/generation_config.json
Model weights saved in ft-smollm-135M-instruct-with-simple-cal/checkpoint-6250/model.safetensors
tokenizer config file saved in ft-smollm-135M-instruct-with-simple-cal/checkpoint-6250/tokenizer_config.json
Special tokens file saved in ft-smollm-135M-instruct-with-simple-cal/checkpoint-6250/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=6250, training_loss=0.2853729229736328, metrics={'train_runtime': 2438.3413, 'train_samples_per_second': 20.506, 'train_steps_per_second': 2.563, 'total_flos': 0.0, 'train_loss': 0.2853729229736328, 'epoch': 1.0})

In [10]:
kwargs={
    'model_name': os.getenv("WANDB_NAME"),
    'finetuned_from': os.getenv('MODEL_NAME'),
#     'tasks': 'Text-Generation',
#     'dataset_tags':'',
    'dataset': os.getenv("DATASET")
}

tokenizer.push_to_hub(os.getenv("WANDB_NAME"))
trainer.push_to_hub(**kwargs)

README.md:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer config file saved in ft-smollm-135M-instruct-with-simple-cal/tokenizer_config.json
Special tokens file saved in ft-smollm-135M-instruct-with-simple-cal/special_tokens_map.json
Uploading the following files to micost/ft-smollm-135M-instruct-with-simple-cal: tokenizer_config.json,tokenizer.json,vocab.json,special_tokens_map.json,merges.txt,README.md
Saving model checkpoint to ft-smollm-135M-instruct-with-simple-cal
Configuration saved in ft-smollm-135M-instruct-with-simple-cal/config.json
Configuration saved in ft-smollm-135M-instruct-with-simple-cal/generation_config.json
Model weights saved in ft-smollm-135M-instruct-with-simple-cal/model.safetensors
tokenizer config file saved in ft-smollm-135M-instruct-with-simple-cal/tokenizer_config.json
Special tokens file saved in ft-smollm-135M-instruct-with-simple-cal/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generatio

events.out.tfevents.1730168797.0fc1bd7735e1.24.0:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/micost/ft-smollm-135M-instruct-with-simple-cal/commit/9f68cbfa4f08173207c592f7e1210ebe9ff79ce5', commit_message='End of training', commit_description='', oid='9f68cbfa4f08173207c592f7e1210ebe9ff79ce5', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
model=AutoModelForCausalLM.from_pretrained(
    os.getenv("WANDB_NAME"), 
    torch_dtype=torch.float16, 
    device_map="cuda", 
    trust_remote_code=True
)

chat=[
    [{"role":"user","content":"{'A':591,'op':'+','B':78}"}],
    [{"role": "user", "content": "{'A':591,'op':'-','B':78}"}],
    [{"role": "user", "content": "{'A':591,'op':'*','B':78}"}],
    [{"role": "user", "content": "{'A':591,'op':'/','B':78}"}],
    [{"role": "user", "content": "{'A':591,'op':'+','B':785}"}],
    [{"role": "user", "content": "{'A':591,'op':'+','B':784}"}]
]


for c in chat:
    p=tokenizer.apply_chat_template(c, tokenize=False)
    inputs = tokenizer(p, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, do_sample=True, pad_token_id=tokenizer.eos_token_id, top_p=0.9, max_new_tokens=150)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(result)

loading configuration file ft-smollm-135M-instruct-with-simple-cal/config.json
Model config LlamaConfig {
  "_name_or_path": "ft-smollm-135M-instruct-with-simple-cal",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 576,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 9,
  "num_hidden_layers": 30,
  "num_key_value_heads": 3,
  "pad_token_id": 2,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "float16",
  "transformers_version": "4.39.3",
  "use_cache": true,
  "vocab_size": 49152
}

loading weights file ft-smollm-135M-instruct-with-simple-cal/model.safetensors
Instantiating LlamaForCausalLM model under default dtype torch.float16.
Generate

user
{'A':591,'op':'+','B':78}
assistant
{'A':591,'op':'+','B':78}
user
{'A':591,'op':'-','B':78}
assistant
{'A':58,'op':'-','B':78}
user
{'A':591,'op':'*','B':78}
assistant
{'A':591,'op':'*','B':78}
user
{'A':591,'op':'/','B':78}
assistant
{'A':591,'op':'/','B':78}
user
{'A':591,'op':'+','B':785}
assistant
{'A':591,'op':'+','B':785}
user
{'A':591,'op':'+','B':784}
assistant
{'A':591,'op':'+','B':78}


# Helpful papers

* https://arxiv.org/abs/2403.07691